In [17]:
import csv
import pandas as pd
import mysql.connector

In [18]:
def upload_csv(input_path, output_path, needed_columns, host, user, password, database, table_name):
    try:
        df = pd.read_csv('june_closed_vuln1.csv')

        df['Month'] = 'June'

        cleaned_df = df[needed_columns]

        cleaned_df.to_csv(output_path, index=False)

        print(f"CSV file cleaned successfully. Saved to {output_path}")

        upload_table(output_path, ',', host, user, password, database, table_name)


    except Exception as e:
        print(f"Error: {e}")

In [19]:
def upload_table(file_path, delim, host, user, password, database, table_name):

    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            auth_plugin='mysql_native_password'
        )

        cursor = connection.cursor()

        with open(file_path,  'r' ) as f:
            reader = csv.reader(f, delimiter=delim)

            next(reader, None)

            for row in reader:
                row = ['NULL' if val == '' else val for val in row]
                row = [x.replace("'", "''") for x in row]
                out = "'" + "', '".join(str(item) for item in row) + "'"
                out = out.replace("'NULL'", 'NULL')

                query = f"INSERT INTO {table_name} VALUES ({out})"
                cursor.execute(query)

        connection.commit()
        connection.close()

        print(f"Uploaded data to table {table_name}.")

    except Exception as e:
        print(f"Error: {e}")

In [20]:
input_csv_path = 'june_closed_vuln1.csv'
output_csv_path = 'cleaned_june_closed.csv'
needed_columns_list = ['Month', 'Server IP', 'Severity', 'Vulnerability Name']

upload_csv(
    input_csv_path,
    output_csv_path,
    needed_columns_list,
    '********', #Host name
    '********', #db name
    '********', #u_name
    '********', #pword
    '**********' #table_name
)

CSV file cleaned successfully. Saved to cleaned_june_closed.csv
Uploaded data to table Closed_vulnerabilities.
